### Web scrapping for [maroof Website]

In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

In [1]:
# import requests

# # API URL for fetching store information
# api_url = "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=5&maxResultCount=5"
# api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"

# # API request headers
# headers = {
#     'apikey': api_key
# }

# def fetch_store_ids(api_url):
#     print(f"Fetching store information from: {api_url}")
    
#     response = requests.get(api_url, headers=headers)
#     print(f"Status Code: {response.status_code}")
    
#     if response.status_code != 200:
#         print("Failed to retrieve data.")
#         return None
    
#     try:
#         data = response.json()
#     except ValueError as e:
#         print(f"Error parsing JSON: {e}")
#         return None
    
#     store_ids = [item.get('id') for item in data.get('items', [])]
#     return store_ids

# # Fetch store IDs
# store_ids = fetch_store_ids(api_url)

# # Print the store IDs
# print("Store IDs:")
# print(store_ids)

# # Optionally, save the store IDs to a file
# # with open('store_ids.txt', 'w') as f:
# #     for store_id in store_ids:
# #         f.write(f"{store_id}\n")


### get store IDs

In [2]:

# API URL for fetching store information
api_url = "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=0&maxResultCount=500"
api_url2 = "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=500&maxResultCount=500"
api_url3 = "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=1000&maxResultCount=500"
api_url4 = "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=1500&maxResultCount=500"
api_url5= "https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=2000&maxResultCount=500"
api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"

# API request headers
headers = {
    'apikey': api_key
}

def fetch_store_ids(api_url):
    print(f"Fetching store information from: {api_url}")
    
    response = requests.get(api_url, headers=headers)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code != 200:
        print("Failed to retrieve data.")
        return None
    
    try:
        data = response.json()
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
    store_ids = [item.get('id') for item in data.get('items', [])]
    return store_ids

def fetch_store_ids_parallel(api_urls):
    store_ids = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(fetch_store_ids, url): url for url in api_urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                if result:
                    store_ids.extend(result)
            except Exception as exc:
                print(f"{url} generated an exception: {exc}")
    return store_ids

# List of API URLs (for demonstration, using the same URL; you might have different URLs)
api_urls = [api_url,
api_url2,
api_url3,
api_url4,
api_url5]  # Example: requesting the same URL multiple times
print(api_urls)
# Fetch store IDs using parallel requests
store_ids = fetch_store_ids_parallel(api_urls)

# Print the store IDs
print("Store IDs:")
print(store_ids)



['https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=0&maxResultCount=500', 'https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=500&maxResultCount=500', 'https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=1000&maxResultCount=500', 'https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=1500&maxResultCount=500', 'https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&

### get reviews from stores

In [1]:

# Base URL for the API endpoint
api_url_template = "https://api.thiqah.sa/maroof/public/api/app/business/{store_id}/reviews?skipCount=0&maxResultCount=10&sortColumn=&sortDirection=2&onlyReviewsWithComments=true"

# API key
api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"
headers = {
    'apikey': f'{api_key}'
}

def get_store_reviews(store_id):
    api_url = api_url_template.format(store_id=store_id)
    print(f"Fetching reviews from: {api_url}")
    
    response = requests.get(api_url, headers=headers)
    print(f"Status Code: {response.status_code}")

    if response.status_code != 200:
        print("Failed to retrieve data.")
        return None
    
    try:
        data = response.json()
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
    reviews = []
    for item in data.get('items', []):
        review = {
            'store_id': store_id,
            'review_id': item.get('id'),
            'rating': item.get('rating'),
            'comment': item.get('comment'),
            'creation_date': item.get('creationDate'),
            'user_name': item.get('userName')
        }
        reviews.append(review)
    
    return reviews

def fetch_reviews_parallel(store_ids):
    all_reviews = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_store_id = {executor.submit(get_store_reviews, store_id): store_id for store_id in store_ids}
        for future in as_completed(future_to_store_id):
            store_id = future_to_store_id[future]
            try:
                result = future.result()
                if result:
                    all_reviews.extend(result)
            except Exception as exc:
                print(f"{store_id} generated an exception: {exc}")
    return all_reviews


# Fetch reviews for each store in parallel
all_reviews = fetch_reviews_parallel(store_ids)

# Convert to DataFrame
df = pd.DataFrame(all_reviews)

# Print the DataFrame
print("DataFrame:")
# print(df)

# Optionally, save to a CSV file


KeyboardInterrupt: 

In [10]:
df


In [ ]:
df.to_csv('store_reviews.csv', index=False)


In [38]:
# import requests 
# import pandas as pd

# # Base URL for the API endpoint
# api_url_template = "https://api.thiqah.sa/maroof/public/api/app/business/{store_id}/reviews?skipCount=0&maxResultCount=10&sortColumn=&sortDirection=2&onlyReviewsWithComments=true"

# # API key
# api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"
# headers = {
#     'apikey': f'{api_key}'
#     }

# def get_store_reviews(store_id):
#     api_url = api_url_template.format(store_id=store_id)
#     print(f"Fetching reviews from: {api_url}")
    
#     response = requests.get(api_url, headers=headers)
#     print(f"Status Code: {response.status_code}")
#     print(f"Response Headers: {response.headers}")
#     print(f"Response Content: {response.content.decode()}")

#     if response.status_code != 200:
#         print("Failed to retrieve data.")
#         return None
    
#     try:
#         data = response.json()
#     except ValueError as e:
#         print(f"Error parsing JSON: {e}")
#         return None
    
#     reviews = []
#     for item in data.get('items', []):
#         review = {
#             'store_id': store_id,
#             'review_id': item.get('id'),
#             'rating': item.get('rating'),
#             'comment': item.get('comment'),
#             'creation_date': item.get('creationDate'),
#             'user_name': item.get('userName')
#         }
#         reviews.append(review)
    
#     return reviews

# # List of store IDs to fetch reviews for

# # Fetch reviews for each store
# all_reviews = []
# for store_id in store_ids:
#     reviews = get_store_reviews(store_id)
#     if reviews:
#         all_reviews.extend(reviews)

# # Convert to DataFrame
# df = pd.DataFrame(all_reviews)

# # Print the DataFrame
# print("DataFrame:")
# df

# # Optionally, save to a CSV file
# # df.to_csv('store_reviews.csv', index=False)


Fetching reviews from: https://api.thiqah.sa/maroof/public/api/app/business/291836/reviews?skipCount=0&maxResultCount=10&sortColumn=&sortDirection=2&onlyReviewsWithComments=true
Status Code: 200
Response Headers: {'Date': 'Tue, 03 Sep 2024 22:06:22 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Connection': 'keep-alive', 'Set-Cookie': 'INGRESSCOOKIE=55199d925a799ca8c55526951ddaf3fe|d2bcfb864db52eb9ef61ad074c2a29bb; Path=/; HttpOnly, TS014c22ab=0142d00c6282ce80d08f5d397dd3fd45c05e0ada74a9f6700155174563656f50be910773dae1314bb49549bfd795300aaf2cbb5ba0d3cdb2214ce40218ea034e1fb232d48d; Path=/, TS0178a7b7=0142d00c623bbe4d73ec9469782d674c922a96e76c13a1a763bd7311090b0750bd13e20817c851db495678a3e89f235fd35058bc5cc800322ed0001466832ed7563a4072c3f4ab083c004ebd0d71da68ec5522b599; Path=/; Domain=.api.thiqah.sa', 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Expires': '0', 'Pragma': 'no-cache', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Frame-O

,store_id,review_id,rating,comment,creation_date,user_name
0,291836,368253,5,تعاملت مع التصميم الفاخر وجدتهم محلات تفصيل ال...,2024-09-02T14:24:57.21,كمال بن سعيد
1,291836,368204,5,وسيطه متعاونه,2024-09-01T03:36:08.177,Lena Zhrani
2,48066,365473,5,متجر مميز,2024-05-19T20:16:47.887,عبدالرحمن محمد
3,48066,210627,5,من المتاجر اللي طلبت منها وماندمت اني طلبت اول...,2021-05-26T12:53:57.367,Malik
4,48066,205322,5,من المتاجر المُحببة للقلب أسلوب وتعامل وطريقك ...,2021-04-30T19:10:13.313,لجين يوسف
...,...,...,...,...,...,...
472,41414,215604,5,تعامل جدا راقي ورحابة صدرها وانسانة ذوق وجدا م...,2021-06-25T18:57:08.11,Hanan Rayes
473,41414,189486,5,تعامل راقي جداً و مصادقية و دقة في المواعيد ❤,2021-02-24T22:57:16.71,Enam Abdulh
474,41414,189482,5,Perfect👌🏻,2021-02-24T22:41:16.82,Marwah Madan
475,41414,187982,5,تعامل راقي جداً🌹,2021-02-18T02:20:44.77,Mashael Assaadi


In [39]:
# df
df.head(50)

# # Optionally, save to a CSV file  apikey
# df.to_csv('store_data.csv', index=False)


,store_id,review_id,rating,comment,creation_date,user_name
0,291836,368253,5,تعاملت مع التصميم الفاخر وجدتهم محلات تفصيل ال...,2024-09-02T14:24:57.21,كمال بن سعيد
1,291836,368204,5,وسيطه متعاونه,2024-09-01T03:36:08.177,Lena Zhrani
2,48066,365473,5,متجر مميز,2024-05-19T20:16:47.887,عبدالرحمن محمد
3,48066,210627,5,من المتاجر اللي طلبت منها وماندمت اني طلبت اول...,2021-05-26T12:53:57.367,Malik
4,48066,205322,5,من المتاجر المُحببة للقلب أسلوب وتعامل وطريقك ...,2021-04-30T19:10:13.313,لجين يوسف
5,48066,204942,5,من افضل المتاجر الي تعاملت معها ، تعامل جداً ر...,2021-04-28T18:07:19.473,Lulu Alaqeel
6,48066,198587,5,متجر جميل والله بكل امانه وصدق مهتمين ب أدق ال...,2021-03-28T20:59:52.74,abeer
7,48066,178465,5,متجر سهل التعامل معاه والخدمه سريعه والطلبية ت...,2020-12-28T10:48:44.993,رنا
8,48066,168450,5,متجر جميل جداً وتعامل راقي جداً وسرعه التوصيل ...,2020-11-12T17:41:55.24,Just Waad
9,48066,165031,5,متجر فخم الصراحة وتعاملهم حلو وسرعه بالتوصيل💞💞,2020-10-28T19:25:02.893,Legend s


In [21]:
# # import requests
# # import pandas as pd
# # from concurrent.futures import ThreadPoolExecutor, as_completed

# # API URLs
# api_urls = []

# for i in range(0, 66001, 500):
#     api_urls.append("https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount={}&maxResultCount=500".format(i))

# # API key and headers
# api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"
# headers = {'apikey': api_key}

# # Fetch store information from API
# def fetch_store_info(api_url):
#     response = requests.get(api_url, headers=headers)
#     if response.status_code == 200:
#         print("Done")
#     else:
#         print(f"Failed to retrieve data from {api_url}")
#         return []
    
#     data = response.json()
#     store_info = []

#     # Extracting information for stores with at least 50 reviews
#     for item in data.get('items', []):
#         if item.get('totalReviews', 0) >= 20:
#             store_info.append({
#                 'Name': item.get('name'),
#                 'NameAr': item.get('nameAr'),
#                 'LocalizedName': item.get('localizedName'),
#                 'BusinessType': item.get('businessType', {}).get('name'),
#                 'OtherTypeName': item.get('otherTypeName'),
#                 'IsPopularBusiness': item.get('isPopularBusiness'),
#                 'TotalReviews': item.get('totalReviews'),
#                 'Rating': item.get('rating'),
#                 'Id': item.get('id'),
#                 'ActiveStatus': item.get('activeStatus'),
#                 'OwnerAllowStatus': item.get('ownerAllowStatus'),
#                 'CertificationStatus': item.get('certificationStatus')
#             })

#     return store_info

# # Fetch store info in parallel
# def fetch_store_info_parallel(api_urls):
#     all_store_info = []
#     with ThreadPoolExecutor(max_workers=20) as executor:
#         future_to_url = {executor.submit(fetch_store_info, url): url for url in api_urls}
#         for future in as_completed(future_to_url):
#             try:
#                 result = future.result()
#                 if result:
#                     all_store_info.extend(result)
#             except Exception as exc:
#                 print(f"Exception occurred: {exc}")
    
#     return all_store_info

# # Fetch store information and put into dataframe
# store_info = fetch_store_info_parallel(api_urls)

# # Convert to DataFrame
# stores_df = pd.DataFrame(store_info)

# # Display the dataframe
# stores_df


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Failed to retrieve data from https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=0&maxResultCount=500
Failed to retrieve data from https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=6500&maxResultCount=500
Failed to retrieve data from https://api.thiqah.sa/maroof/public/api/app/business/search?keyword=&businessTypeId=&businessTypeSubCategoryId=&regionId=&cityId=&certificationType=&sortBy=2&sortDirection=2&sorting=&skipCount=7500&maxResultCount=500
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


,Name,NameAr,LocalizedName,BusinessType,OtherTypeName,IsPopularBusiness,TotalReviews,Rating,Id,ActiveStatus,OwnerAllowStatus,CertificationStatus
0,zKcM,زكسم,زكسم,خدمات الأعمال,None,False,31,1.58,232851,0,1,1
1,qasralawani,قصر الأواني,قصر الأواني,خدمات الأعمال,None,False,61,1.46,96843,0,1,1
2,LAVERNE PERFUMES,لافيرن للعطور,لافيرن للعطور,أخرى,None,False,20,1.45,259407,0,1,1
3,Sheta and Saif,الشتاء والصيف,الشتاء والصيف,إلكترونيات,None,False,45,1.41,20209,0,1,1
4,IBRAHEEM ALQURASHI,إبراهيم القرشي,إبراهيم القرشي,خدمات الأعمال,None,False,29,1.28,86363,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
252,Hamtaro STORE,متجر همتارو الإلكتروني,متجر همتارو الإلكتروني,خدمات الأعمال,None,False,60,4.52,231548,0,1,2
253,FINEST JEWELRY,فاينست جولري لتصميم و صناعة المجوهرات,فاينست جولري لتصميم و صناعة المجوهرات,الجمال والصحة,صناعة المعادن الثمينة والاحجار الكريمة,False,42,4.52,2106,0,1,1
254,My mobile accessories store,متجر اكسسوارات جوالي,متجر اكسسوارات جوالي,إلكترونيات,None,False,29,4.52,180632,0,1,1
255,CARDKSA STORE,متجر كارد السعودية الرسمي,متجر كارد السعودية الرسمي,إلكترونيات,بطاقات وإشتركات رقمية,False,25,4.52,71456,0,1,2


In [19]:
stores_df.shape

(66400, 12)

In [20]:
stores_df.to_csv('all_stores.csv', index=False)


In [22]:
df1 = pd.read_csv('all_stores.csv')


,Name,NameAr,LocalizedName,BusinessType,OtherTypeName,IsPopularBusiness,TotalReviews,Rating,Id,ActiveStatus,OwnerAllowStatus,CertificationStatus
0,KidsOasis,واحة الأطفال,واحة الأطفال,إلكترونيات,NaN,False,1,5.0,318150,0,1,2
1,Hanat ajdadna,حنة اجدادنا,حنة اجدادنا,الجمال والصحة,NaN,False,1,5.0,318121,0,1,2
2,ARIS ALHALLOUL MOTAKAMLA Limited liability,شركة اريس الحلول المتكاملة المحدودة,شركة اريس الحلول المتكاملة المحدودة,أخرى,خدمات السفر السياحة,False,1,5.0,318099,0,1,1
3,ALTADAWOL ALEHTERAFY,التداول الإحترافي,التداول الإحترافي,التعليم,NaN,False,1,5.0,318090,0,1,2
4,Bankai Store,متجر بانكاي,متجر بانكاي,إلكترونيات,NaN,False,1,5.0,318087,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
66395,NaN,متجر انبوب الالكتروني,متجر انبوب الالكتروني,أخرى,NaN,False,0,0.0,121104,0,1,1
66396,raheebx,مؤسسة ابداعات رهيب للاتصالات,مؤسسة ابداعات رهيب للاتصالات,إلكترونيات,NaN,False,0,0.0,121097,0,1,1
66397,Datesgardenn,مؤسسة بساتين التمور,مؤسسة بساتين التمور,خدمات الأعمال,NaN,False,0,0.0,121085,0,1,1
66398,gifts Afnan,متجر هدايا أفنان,متجر هدايا أفنان,أخرى,NaN,False,0,0.0,121078,0,1,2


In [50]:
df50 = df1[(df1['TotalReviews'] >=50)]

In [53]:
df50['TotalReviews'].sort_values(ascending=False).sum()

47177

In [79]:

# Base URL for the API endpoint
api_url_template = "https://api.thiqah.sa/maroof/public/api/app/business/{store_id}/reviews?skipCount=0&maxResultCount=10000&sortColumn=&sortDirection=2&onlyReviewsWithComments=false"

# API key
api_key = "c1qesecmag8GSbxTHGRjfnMFBzAH7UAN"
headers = {
    'apikey': f'{api_key}'
}

def get_store_reviews(store_id):
    api_url = api_url_template.format(store_id=store_id)
    
    response = requests.get(api_url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve data.")
        return None
    
    try:
        data = response.json()
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
    reviews = []
    for item in data.get('items', []):
        review = {
            'BusinessId': store_id,
            'review_id': item.get('id'),
            'rating': item.get('rating'),
            'comment': item.get('comment'),
            'creation_date': item.get('creationDate')
        }
        reviews.append(review)
    
    return reviews

def fetch_reviews_parallel(store_ids):
    all_reviews = []
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_store_id = {executor.submit(get_store_reviews, store_id): store_id for store_id in store_ids}
        for future in as_completed(future_to_store_id):
            store_id = future_to_store_id[future]
            try:
                result = future.result()
                if result:
                    all_reviews.extend(result)
            except Exception as exc:
                print(f"{store_id} generated an exception: {exc}")
    return all_reviews


# Fetch reviews for each store in parallel
all_reviews = fetch_reviews_parallel(list(df50['Id'].values))

# Convert to DataFrame
df_reviews = pd.DataFrame(all_reviews)

# Print the DataFrame
print("DataFrame:")
# print(df)
df_reviews
# Optionally, save to a CSV file


DataFrame:


,BusinessId,review_id,rating,comment,creation_date
0,268517,368276,5,تعامل راقي ومتعاونين جدا يعطيكم العافيه والطلب...,2024-09-03T10:46:56.273
1,268517,366180,5,التعامل يفتح النفس ويسون نفس المطلوب والتوصيل ...,2024-06-10T17:02:09.137
2,268517,366102,5,اسلوب وذوق ماشاءالله,2024-06-07T19:02:15.573
3,268517,361135,5,تعامل رائع ان شاء الله مب اخر مره اطلب منها,2024-05-08T15:44:14.163
4,268517,359092,5,اشكرهم على حسن التعامل والاسلوب والتنسيق الحلو...,2024-04-03T21:42:03.303
...,...,...,...,...,...
47172,277496,326234,5,,2023-03-30T20:29:55.22
47173,277496,326233,5,,2023-03-30T20:29:52.063
47174,277496,326232,5,,2023-03-30T20:29:49.333
47175,277496,326231,5,,2023-03-30T20:29:46.557


In [81]:
df_reviews['BusinessId'].value_counts(ascending=False)

BusinessId
39976     4596
77092     3333
48246     1813
24519     1553
257509    1204
          ... 
254977      50
27286       50
107514      50
121265      50
268517      50
Name: count, Length: 213, dtype: int64

In [82]:
df_reviews.to_csv('reviews.csv')